In [0]:
from google.colab import drive

drive.mount('/content/drive')

import os 

os.chdir("/content/drive/My Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Adapted from https://github.com/Hzzone/pytorch-openpose

import torch
import torch.nn as nn
from collections import OrderedDict

def constructing_block(block, not_relu_layer):
    layers = []
    for layer_name, v in block.items():
        #Pooling layer
        if 'pool' in layer_name:
            layer = nn.MaxPool2d(kernel_size=v[0], stride=v[1], padding=v[2])
            layers.append((layer_name, layer))
        
        #Convolution layer
        else:
            conv2d = nn.Conv2d(in_channels=v[0], out_channels=v[1],
                               kernel_size=v[2], stride=v[3],
                               padding=v[4])
            layers.append((layer_name, conv2d))
            
            #Add RELU
            if layer_name not in not_relu_layer:
                layers.append(('relu_'+layer_name, nn.ReLU(inplace=True)))

    return nn.Sequential(OrderedDict(layers))

class Pose_Net(nn.Module):
    def __init__(self):
        super(Pose_Net, self).__init__()

        # Names of layers with no RELU activation
        no_relu_layers = ['conv5_5_CPM_L1', 'conv5_5_CPM_L2', 'Mconv7_stage2_L1',\
                          'Mconv7_stage2_L2', 'Mconv7_stage3_L1', 'Mconv7_stage3_L2',\
                          'Mconv7_stage4_L1', 'Mconv7_stage4_L2', 'Mconv7_stage5_L1',\
                          'Mconv7_stage5_L2', 'Mconv7_stage6_L1', 'Mconv7_stage6_L1']
        blocks = {}


        #First stage: detecting keypoints
        
        self.block0 = constructing_block(OrderedDict({'conv1_1': [3, 64, 3, 1, 1],                      #[input size, output size, kernel size, stride, padding]
                                                      'conv1_2': [64, 64, 3, 1, 1],
                                                      'pool1_kp': [2, 2, 0],                            #[kernel size, stride, padding]
                                                      'conv2_1': [64, 128, 3, 1, 1],
                                                      'conv2_2': [128, 128, 3, 1, 1],
                                                      'pool2_kp': [2, 2, 0],
                                                      'conv3_1': [128, 256, 3, 1, 1],
                                                      'conv3_2': [256, 256, 3, 1, 1],
                                                      'conv3_3': [256, 256, 3, 1, 1],
                                                      'conv3_4': [256, 256, 3, 1, 1],
                                                      'pool3_kp': [2, 2, 0],
                                                      'conv4_1': [256, 512, 3, 1, 1],
                                                      'conv4_2': [512, 512, 3, 1, 1],
                                                      'conv4_3_CPM': [512, 256, 3, 1, 1],
                                                      'conv4_4_CPM': [256, 128, 3, 1, 1]}), no_relu_layers)
        
        self.block1_1 = constructing_block(OrderedDict({'conv5_1_CPM_L1': [128, 128, 3, 1, 1],
                                                        'conv5_2_CPM_L1': [128, 128, 3, 1, 1],
                                                        'conv5_3_CPM_L1': [128, 128, 3, 1, 1],
                                                        'conv5_4_CPM_L1': [128, 512, 1, 1, 0],
                                                        'conv5_5_CPM_L1': [512, 38, 1, 1, 0]}), no_relu_layers)

        self.block1_2 = constructing_block(OrderedDict({'conv5_1_CPM_L2': [128, 128, 3, 1, 1],
                                                        'conv5_2_CPM_L2': [128, 128, 3, 1, 1],
                                                        'conv5_3_CPM_L2': [128, 128, 3, 1, 1],
                                                        'conv5_4_CPM_L2': [128, 512, 1, 1, 0],
                                                        'conv5_5_CPM_L2': [512, 19, 1, 1, 0]}), no_relu_layers)

        # Second stage: inferring individuals
        for i in range(2, 7):
            blocks['block%d_1' % i] = OrderedDict({
                                'Mconv1_stage%d_L1' % i: [185, 128, 7, 1, 3],
                                'Mconv2_stage%d_L1' % i: [128, 128, 7, 1, 3],
                                'Mconv3_stage%d_L1' % i: [128, 128, 7, 1, 3],
                                'Mconv4_stage%d_L1' % i: [128, 128, 7, 1, 3],
                                'Mconv5_stage%d_L1' % i: [128, 128, 7, 1, 3],
                                'Mconv6_stage%d_L1' % i: [128, 128, 1, 1, 0],
                                'Mconv7_stage%d_L1' % i: [128, 38, 1, 1, 0]})

            blocks['block%d_2' % i] = OrderedDict({
                                'Mconv1_stage%d_L2' % i: [185, 128, 7, 1, 3],
                                'Mconv2_stage%d_L2' % i: [128, 128, 7, 1, 3],
                                'Mconv3_stage%d_L2' % i: [128, 128, 7, 1, 3],
                                'Mconv4_stage%d_L2' % i: [128, 128, 7, 1, 3],
                                'Mconv5_stage%d_L2' % i: [128, 128, 7, 1, 3],
                                'Mconv6_stage%d_L2' % i: [128, 128, 1, 1, 0],
                                'Mconv7_stage%d_L2' % i: [128, 19, 1, 1, 0]})

        for k in blocks.keys():
            blocks[k] =constructing_block(blocks[k], no_relu_layers)

        self.block2_1 = blocks['block2_1']
        self.block3_1 = blocks['block3_1']
        self.block4_1 = blocks['block4_1']
        self.block5_1 = blocks['block5_1']
        self.block6_1 = blocks['block6_1']

        self.block2_2 = blocks['block2_2']
        self.block3_2 = blocks['block3_2']
        self.block4_2 = blocks['block4_2']
        self.block5_2 = blocks['block5_2']
        self.block6_2 = blocks['block6_2']


    def forward(self, x):

        out1 = self.block0(x)

        out1_1 = self.block1_1(out1)
        out1_2 = self.block1_2(out1)
        out2 = torch.cat([out1_1, out1_2, out1], 1)

        out2_1 = self.block2_1(out2)
        out2_2 = self.block2_2(out2)
        out3 = torch.cat([out2_1, out2_2, out1], 1)

        out3_1 = self.block3_1(out3)
        out3_2 = self.block3_2(out3)
        out4 = torch.cat([out3_1, out3_2, out1], 1)

        out4_1 = self.block4_1(out4)
        out4_2 = self.block4_2(out4)
        out5 = torch.cat([out4_1, out4_2, out1], 1)

        out5_1 = self.block5_1(out5)
        out5_2 = self.block5_2(out5)
        out6 = torch.cat([out5_1, out5_2, out1], 1)

        out6_1 = self.block6_1(out6)
        out6_2 = self.block6_2(out6)

        return out6_1, out6_2

In [0]:
# Load Packages
import numpy as np 
import matplotlib.pyplot as plt 
from glob import glob
import cv2
import os
from PIL import Image
from io import BytesIO
from torch.utils.data import Dataset
from sklearn.cluster import KMeans
import skimage.segmentation as ski
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import torch.utils.data as utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as opt
import matplotlib.pyplot as plt
# from tensorboardX import SummaryWriter
import subprocess
from sklearn.model_selection import train_test_split
import sys

In [0]:
# Set image size
IMG_SIZE = (256, 256)


'''
CREATE THE DATA
'''

class NumPyDataset(Dataset):
    """ Creates a Pytorch Dataset out of Numpy arrays. """
    def __init__(self, X, y, transform_img=None, transform_mask=None):
        self.X = X
        self.y = y
        self.transform_img = transform_img
        self.transform_mask = transform_mask
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        # Turn arrays into images
        X_sample = Image.fromarray((self.X[index] * 255).astype(np.uint8))
        y_sample = Image.fromarray((self.y[index]).astype(np.uint8))
        # Transform images if necessary
        X_sample = self.transform_img(X_sample)
        y_sample = self.transform_mask(y_sample)
        return X_sample, y_sample

def create_random_transformation(n):
    transformations_img, transformations_mask = [], []
    # Identify ranges of possible parameter values
    brightness = np.linspace(0, 0.2, 10)
    contrast = np.linspace(0, 0.2, 10)
    saturation = np.linspace(0, 0.2, 10)
    hue = np.linspace(0, 0.2, 10)
    hflip = np.array([0, 1])
    vflip = np.array([0, 1])
    rotate = np.linspace(0, 360, 36)
    
    for _ in range(n):
        # Select specific parameter values for images and GTs
        b = np.random.choice(brightness)
        c = np.random.choice(contrast)
        s = np.random.choice(saturation)
        hu = np.random.choice(hue)
        h = np.random.choice(hflip)
        v = np.random.choice(vflip)
        r = np.random.choice(rotate)
        # Transform for images
        transform_img = transforms.Compose([
            transforms.ColorJitter(brightness=b, contrast=c, saturation=s, hue=hu), 
            transforms.RandomHorizontalFlip(p=h),
            transforms.RandomVerticalFlip(p=v),
            transforms.RandomRotation(degrees=(r,r)),
            transforms.ToTensor(),
        ])
        transformations_img.append(transform_img)
        
        # Transform for GTs
        transform_mask = transforms.Compose([
            transforms.RandomHorizontalFlip(p=h),
            transforms.RandomVerticalFlip(p=v),
            transforms.RandomRotation(degrees=(r,r)),
            transforms.ToTensor(),
        ])
        transformations_mask.append(transform_mask)
    
    return transformations_img, transformations_mask


def create_loaders(img, labels, n_augmented):
    transformations_img, transformations_mask = create_random_transformation(n_augmented)
    org_train_img, org_val_img, org_train_labels, org_val_labels = train_test_split(img,
                                    labels,
                                    test_size=train_val_split) 
    org_train = NumPyDataset(org_train_img, 
                             org_train_labels, 
                             transform_img=transforms.ToTensor(), 
                             transform_mask=transforms.ToTensor()) 
    augmented_train = [NumPyDataset(org_train_img, 
                                    org_train_labels, 
                                    transform_img=trans_img, 
                                    transform_mask=trans_mask)\
                     for trans_img, trans_mask in zip(transformations_img, transformations_mask)]
    augmented_train.append(org_train)
    train_dataset = utils.ConcatDataset(augmented_train)
    
    org_val = NumPyDataset(org_val_img, org_val_labels, transform_img=transforms.ToTensor(), transform_mask=transforms.ToTensor()) 
    
    train_loader = utils.DataLoader(train_dataset, batch_size=batch_size)
    validation_loader = utils.DataLoader(org_val, batch_size=batch_size)
    return train_loader, validation_loader